In [110]:
import pandas as pd
import numpy as np
from itertools import cycle 
import seaborn as sns
import matplotlib.pyplot as plt 

vehicles = pd.read_csv("/Users/avanelson/Desktop/IST652 Python/Python Project/vehicles_trimmed_coupe.csv")
#///////import data////////

type_dict = {
    'price': float,
    'year': 'Int64',
    'manufacturer': str,
    'model': str,
    'condition': str,
    'cylinders': str,
    'fuel': str,
    'odometer': 'Int64',
    'title_status': str,
    'transmission': str,
    'size': str,
    'type': str,
    'paint_color': str,
    'state': str,
}
vehicles = vehicles.astype(type_dict)
vehicles.dtypes #verify data types

'''Import dataset for cleaning up car make/model listing for easier processing'''
make_model = pd.read_csv("/Users/avanelson/Desktop/IST652 Python/Python Project/make_model.csv", sep=";", usecols=[0,1]).apply(lambda x: x.astype(str).str.lower()) #source credit: http://tiresaddict.com/help/databases/cars/
make_model.columns = ['Make', 'Model'] #rename columns
model_list = make_model[['Model']].to_numpy().tolist()

'''ISOLATE MODEL LIST WITH MODELS THAT EXIST IN DATAFRAME'''
vehicle_list = vehicles[['model']].to_numpy().tolist()
vehicle_list = [str(i) for i in vehicle_list]
model_list = [str(i) for i in model_list]

def checkIfValuesExists1(dfObj, listOfValues):
    ''' Check if given elements exists in dictionary or not.
        It returns a dictionary of elements as key and thier existence value as bool'''
    resultDict = {}
    # Iterate over the list of elements one by one
    for elem in listOfValues:
        # Check if the element exists in dataframe values
        if elem in dfObj:
            resultDict[elem] = True
        else:
            resultDict[elem] = False
    # Returns a dictionary of values & thier existence flag        
    return resultDict

models_TF = checkIfValuesExists1(vehicle_list, model_list) #Run Function

'''TRIM DOWN LIST BY REMOVING FALSE VARIABLES'''
def filterTheDict(dictObj, callback):
    newDict = dict()
    # Iterate over all the items in dictionary
    for (key, value) in dictObj.items():
        # Check if item satisfies the given condition then add to new dict
        if callback((key, value)):
            newDict[key] = value
    return newDict

f_Models_dict = filterTheDict(models_TF, lambda elem: elem[1] == True)

filtered_Models = list(f_Models_dict.keys())
import re

filtered_Models_2 = []
count = 0
for i in filtered_Models:
    stripped = re.sub('[\W_]+', '', i)
    filtered_Models_2.append(stripped) #clear out non-alphanumeric characters




In [135]:
'''RUN NEW MODEL DICTIONARY AGAINST CAR MODELS'''
old_model = []
simp_model = []
old_new_model = {}
model_index = []
count = 0

for model in vehicles.model:
    model_index = filtered_Models_2[count]
    old_model = [s for s in vehicles.model if model_index in s] #match df with entry from model list
    simp_model = model_index
    extra = dict(zip(old_model, [model_index])) #make dictionary with old-new entries
    old_new_model.update(extra)
    count = count + 1
        
    

    

In [150]:
'''NORMALIZE MODEL NAMES'''

#vehicles_filtered = vehicles[vehicles['model'].isin(old_new_model.keys())] #filter out rows not present in dict
#vehicles_filtered.model = vehicles_filtered['model'].map(old_new_model) #replace with normalized model names



/Users/avanelson/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [291]:
len(make_model)

1121

In [209]:
'''REPLACE NaN MANUF NAMES'''

df_findManuf = vehicles_filtered[vehicles_filtered['manufacturer'] == 'nan']
#filter out just the nan values

make_model_filtMake = make_model[make_model['Model'].isin(df_findManuf.model.unique())]
#filter out makes needed to supply for nan

for make, model in make_model_filtMake.itertuples(index=False):
    df_findManuf['manufacturer'] = np.where((df_findManuf.model == model), make, df_findManuf.manufacturer)
    
vehicles_filtered.update(df_findManuf) #update missing values with values
vehicles_filtered.to_csv(r'/Users/avanelson/Desktop/vehicles_final.csv', header=True)

In [148]:
'''BEGIN EXPLORATORY ANALYSIS'''

'BEGIN EXPLORATORY ANALYSIS'

In [207]:
#Q1. min/max/avg price, year, odometer for each car brand.
manuf = vehicles_filtered.manufacturer.unique()
ManufList = {}
count = 0
for item in vehicles_filtered:
    df1 = vehicles_filtered[vehicles_filtered['manufacturer'] == manuf[count]]
    
    MinPrice = min(df1['price'].dropna())
    MaxPrice = max(df1['price'].dropna())
    AvgPrice = df1['price'].dropna().mean()
    
    MinYear = min(df1['year'].dropna())
    MaxYear = max(df1['year'].dropna())
    AvgYear = int(df1['year'].dropna().mean())
    
    MinOdo = min(df1['odometer'].dropna())
    MaxOdo = max(df1['odometer'].dropna())
    AvgOdo = int(df1['odometer'].dropna().mean())
    
    #add calculations as new dict entry
    ManufList[manuf[count]] = [MinYear, AvgYear, MaxYear, MinPrice, AvgPrice, MaxPrice, MinOdo, AvgOdo, MaxOdo]
    count = count + 1
    
BrandStats = pd.DataFrame.from_dict(ManufList, orient='index')
BrandStats.columns = ['Oldest Year', 'Avg Year', 'Newest Year', 'Cheapest', 'Mid Range', 'Most Expensive', 'Least Miles', 'Avg Mileage', 'Most Miles']

#BrandStats.to_csv(r'/Users/avanelson/Desktop/BrandStats.csv', header=True)
BrandStats)

,Oldest Year,Avg Year,Newest Year,Cheapest,Mid Range,Most Expensive,Least Miles,Avg Mileage,Most Miles
ford,1934,1995,2019,0.0,11796.394531,72980.0,101,124326,2000000
bmw,1996,2010,2019,0.0,24734.557143,69999.0,5161,53807,179000
chevrolet,1958,2009,2019,0.0,16735.661258,70000.0,0,62227,510000
honda,1981,2008,2020,0.0,6854.901163,24977.0,0,111968,1493000
hyundai,2000,2014,2017,0.0,8403.660131,15850.0,0,62193,175431
nissan,1990,2006,2018,0.0,12798.513158,135000.0,9479,80587,243443
audi,1999,2011,2018,700.0,38359.913793,147995.0,3650,59185,193294
mini,2008,2011,2019,2800.0,7794.428571,17995.0,29344,97007,147928
toyota,1977,2002,2020,0.0,5677.616438,31500.0,6581,132927,288000
lexus,1970,2007,2017,2995.0,30642.536232,87500.0,22076,61357,256000


In [232]:
#Q1. min/max/avg price, year, odometer for each car model.
models = vehicles_filtered.model.unique()
ModelList = {}
count = 0
for item in vehicles_filtered:
    df1 = vehicles_filtered[vehicles_filtered['model'] == models[count]]
    
    Manuf = df1['manufacturer'].unique()
    
    Qty = len(df1)
    
    MinPrice = min(df1['price'].dropna())
    MaxPrice = max(df1['price'].dropna())
    AvgPrice = df1['price'].dropna().mean()
    
    MinYear = min(df1['year'].dropna())
    MaxYear = max(df1['year'].dropna())
    AvgYear = int(df1['year'].dropna().mean())
    
    MinOdo = min(df1['odometer'].dropna(), default=0) 
    MaxOdo = max(df1['odometer'].dropna(), default=0)
    AvgOdo = df1['odometer'].dropna().mean()
    
    #add calculations as new dict entry
    ModelList[models[count]] = [Manuf, Qty, MinYear, AvgYear, MaxYear, MinPrice, AvgPrice, MaxPrice, MinOdo, AvgOdo, MaxOdo]
    count = count + 1
    
ModelStats = pd.DataFrame.from_dict(ModelList, orient='index')
ModelStats.columns = ['Make', 'Quantity', 'Oldest Year', 'Avg Year', 'Newest Year', 'Cheapest', 'Mid Range', 'Most Expensive', 'Least Miles', 'Avg Mileage', 'Most Miles']

ModelStats.to_csv(r'/Users/avanelson/Desktop/ModelStats.csv', header=True)
ModelStats

,Make,Quantity,Oldest Year,Avg Year,Newest Year,Cheapest,Mid Range,Most Expensive,Least Miles,Avg Mileage,Most Miles
mustang,[ford],117,1964,2003,2019,0.0,9998.666667,32000.0,101,86051.166667,208000
m3,[bmw],5,2011,2011,2011,26500.0,26500.000000,26500.0,54595,54595.000000,54595
coupe,[chevrolet],39,1965,1992,2016,5300.0,18272.205128,70000.0,70,83801.966667,166000
civic,[honda],433,1989,2008,2020,0.0,6470.572748,24977.0,0,109532.061008,500000
crz,[honda],6,2011,2012,2016,4950.0,6841.666667,10000.0,46,62274.833333,115803
veloster,[hyundai],139,2012,2014,2017,0.0,8552.985612,15850.0,0,60852.914062,144195
2,[nissan],1,1996,1996,1996,37800.0,37800.000000,37800.0,213000,213000.000000,213000
a5,[audi],1,2014,2014,2014,16000.0,16000.000000,16000.0,48000,48000.000000,48000
accord,[honda],51,1998,2011,2017,1200.0,11167.294118,20990.0,19436,118067.980000,1493000
cooper,[mini],5,2011,2013,2019,10750.0,13538.800000,17995.0,36741,63176.750000,84983


In [292]:
vehicles_filtered[vehicles_filtered['manufacturer']=='audi']

,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,size,type,paint_color,state
9,16000.0,2014,audi,a5,good,4 cylinders,gas,48000,clean,automatic,compact,coupe,grey,fl
16,13495.0,2012,audi,s5,like new,6 cylinders,gas,<NA>,clean,automatic,mid-size,convertible,white,fl
433,6990.0,2006,audi,a4,excellent,6 cylinders,gas,79354,clean,automatic,compact,convertible,grey,or
1031,57999.0,2009,audi,r8,excellent,8 cylinders,gas,62000,clean,other,nan,coupe,silver,or
1090,6995.0,2006,audi,s4,nan,nan,gas,105785,clean,nan,sub-compact,convertible,silver,fl
1265,22995.0,2017,audi,a3,nan,nan,gas,42162,clean,nan,sub-compact,convertible,black,ma
1763,16999.0,2013,audi,a7,like new,6 cylinders,gas,122100,clean,automatic,full-size,coupe,blue,ma
1932,17950.0,2015,audi,a3,excellent,4 cylinders,gas,<NA>,clean,automatic,compact,convertible,white,ga
1957,146900.0,2017,audi,r8,like new,10 cylinders,gas,9977,clean,automatic,nan,coupe,grey,az
2154,127992.0,2018,audi,r8,nan,nan,gas,6889,clean,automatic,nan,coupe,white,mi
